In [ ]:
import uproot
from matplotlib.colors import LogNorm
import matplotlib.colors as mcolors
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.constants import epsilon_0, e as q_e

from matplotlib.colors import Normalize
from matplotlib.colorbar import ColorbarBase

import glob
import os

import trimesh
from trimesh.points import PointCloud

from common_functions import *

# Results for runs with 1 million particles 

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

# get a single cross section of the mesh
section = stacked_spheres.section(plane_origin=stacked_spheres.centroid, plane_normal=[0, 1, 0])
slice_2D, to_3D = section.to_2D()

fig, ax = plt.subplots()
offset = 100 + 37
ax.axvline(x=-0.16 - offset/1000)
ax.axhline(y=0)
slice_2D.show()
print(f"(X,Y,Z) = ({-0.16 - offset/1000}, 0, -0.05)")


In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

# get a single cross section of the mesh
section = stacked_spheres.section(plane_origin=stacked_spheres.centroid, plane_normal=[1, 0, 0])
slice_2D, to_3D = section.to_2D()

fig, ax = plt.subplots()
offset = 100 + 37
ax.axhline(y=0)
ax.axvline(x=0.168+offset/1000)
plt.xlabel("y-axis")
plt.ylabel("z-axis")
slice_2D.show()
print(f"(X,Y,Z) = ({0.168+offset/1000}, 0, -0.05)")

In [ ]:
# Configuration: choose one
config = "allparticles"  # or "onlysolarwind", "allparticles", "onlyphotoemission"

# Set selectnum 
selectnum = 1000000

# Define directory and file list
directory_path = "../build-sphere-charging/root/"
#directory_path = "/storage/coda1/p-zjiang33/0/shared/avira7/root_files/stacked-sphere/output092425/"
filelist = sorted(glob.glob(f"{directory_path}/*stackediteration0*_num{selectnum}.root"))

# Process each ROOT file
for fileIN in filelist:
    filename = os.path.basename(fileIN)
    
    print(filename)

    # Extract iteration number
    number_str = filename.split("_")[1]
    iterationNUM = int(''.join(filter(str.isdigit, number_str)))

    # Extract config from filename (3rd part after split by "_")
    config_from_file = filename.split("_")[2]

    # Read data
    vars()["df_"+config_from_file] = read_rootfile(filename, directory_path=directory_path)
    _ = calculate_stats(vars()["df_"+config_from_file], config=config_from_file)
    print("df_"+config_from_file)

    print("-" * 78)


In [ ]:
df_onlyphotoemission[df_onlyphotoemission["Parent_ID"]==4.0]

In [ ]:
df_onlyphotoemission[df_onlyphotoemission["Event_Number"]==699633]

In [ ]:
666620/50432

In [ ]:
89000/41979

In [ ]:
# relative intensity for all particles case
4985/87 # want this value to be around 20

# adjust total number of particles so that there are ~81 incident protons for the all particle case

# Check only all particle case (e-, protons, photons)

In [ ]:
# check the input distribution for the photons
input_dist = pd.read_csv("../build-sphere-charging/photon_distribution.txt",header=None,delimiter=" ")

incident_gamma = df_allparticles[df_allparticles["Particle_Type"] == "gamma"].drop_duplicates(subset="Event_Number", keep="first")
plt.hist(incident_gamma["Kinetic_Energy_Pre_MeV"]*1e6,bins=np.logspace(0,5,100),density=True)
print("# of incident gammas: ", len(incident_gamma))
plt.loglog(input_dist[0]*1e6,input_dist[1]/np.max(input_dist[1]),'r-') #*max(histdata[0])
#plt.ylim([1e-5,0.6])
plt.xlabel("Photon Energy (eV)")
plt.yscale("log")
plt.xscale("log")
#plt.xlim([0,100])

In [ ]:
# Create a scene with both mesh and points
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 
# scene = trimesh.Scene([stacked_spheres,calculation_point])

# Coordinates for your "points"
point_coords = np.array([
    [0.03, -0.1, 0.227] #[0,-0.1, 0.167+0.037] #0.27-0.1+0.037] #-0.05,-0.1
])
print([0,-0.1, 0.27-0.1+0.037])
# Create small spheres at each point
spheres = []
for point in point_coords:
    sphere = trimesh.creation.icosphere(radius=0.01)  # adjust radius for point size
    sphere.apply_translation(point)
    sphere.visual.face_colors = [255, 0, 0, 255]  # red spheres
    spheres.append(sphere)

# Combine all meshes into a scene
scene = trimesh.Scene([stacked_spheres] + spheres)


# Show scene in interactive viewer
scene.show()

In [ ]:
gamma_holes0, photoelectrons_inside0, protons_inside0, electron_inside0  = calculate_stats(df_allparticles, config="allparticles")

# plot location of where all the ion stopped in the geometry

# Create a PointCloud object
gamma_photoemission_sites = PointCloud(np.array(protons_inside0["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA red points
photoelectron_stopping_sites = PointCloud(np.array(electron_inside0["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA red points

# Create a scene with both mesh and points
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 
scene = trimesh.Scene([stacked_spheres, gamma_photoemission_sites, photoelectron_stopping_sites])

# Show scene in interactive viewer
scene.show()

In [ ]:
# plot the surface potential of the photoemission
surface,potential = plot_surface_potential_allparticle_case(gamma_holes0, photoelectrons_inside0, protons_inside0, electron_inside0, 
                                                            stacked_spheres, vmin=-3, vmax=3)

print(min(potential), max(potential))
plt.plot(potential[potential!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

# a lot of the surface charge ended up cancelling each other out...

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-3, vmax=3)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
indicent_gamma = df_allparticles[(df_allparticles["Particle_Type"]=="gamma") & (df_allparticles["Parent_ID"]==1.0) & \
                (df_allparticles["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

indicent_electrons = df_allparticles[(df_allparticles["Particle_Type"]=="e-") & (df_allparticles["Parent_ID"]==1.0) &\
                (df_allparticles["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

indicent_protons = df_allparticles[(df_allparticles["Particle_Type"]=="proton") & (df_allparticles["Parent_ID"]==1.0) & \
                (df_allparticles["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([indicent_gamma, indicent_electrons, indicent_protons], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=200)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()

surface_edited.show()

# Check only SW case

In [ ]:
# check the input distribution for the electrons
energy_values = np.linspace(0,50)
histdata = plt.hist(df_onlysolarwind[df_onlysolarwind["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")["Kinetic_Energy_Pre_MeV"]*1e6,bins=100,density=True)
electron_distribution = np.exp(-energy_values**2/10**2)*max(histdata[0][10:])
plt.xlim([-2,30])
plt.ylim([1e-5,0.5])
plt.plot(energy_values, electron_distribution, "r-")
plt.yscale("log")

In [ ]:
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 

In [ ]:
# plot location of where all the elctrons & ion stopped in the geometry

last_protons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"] == "proton")].drop_duplicates(subset="Event_Number", keep="last")
protons_inside = last_protons[(last_protons["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")]
print(len(protons_inside))

last_electrons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"] == "e-")].drop_duplicates(subset="Event_Number", keep="last")
electrons_inside = last_electrons[(last_electrons["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")]
print(len(electrons_inside))

# Create a PointCloud object
proton_stopping_sites = PointCloud(np.array(protons_inside["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA red points
electron_stopping_sites = PointCloud(np.array(electrons_inside["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA red points

# Create a scene with both mesh and points
scene = trimesh.Scene([stacked_spheres, proton_stopping_sites,electron_stopping_sites])

# # apply this transformation to make the orientation match the Zimmerman paper
# T = np.array([
#     [-1, 0,  0, 0],
#     [ 0, 1,  0, 0],
#     [ 0, 0,  1, 0],
#     [ 0, 0,  0, 1]
# ])
# scene = trimesh.Scene()
# scene.add_geometry(stacked_spheres, transform=T)
# scene.add_geometry(proton_stopping_sites, transform=T)
# scene.add_geometry(electron_stopping_sites, transform=T)


# Show scene in interactive viewer
scene.show()

In [ ]:
# plot location of where all the ion stopped in the geometry

surface,potential = plot_surface_potential_fornegativepositive_charge(electrons_inside, protons_inside, stacked_spheres, vmin=-4,vmax=4)
print(min(potential), max(potential))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-1, vmax=1)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
surface,potential = plot_surface_potential_fornegativepositive_charge(electrons_inside, protons_inside, stacked_spheres, vmin=-1,vmax=1)
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None

bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()


In [ ]:
indicent_electrons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"]=="e-") & (df_onlysolarwind["Parent_ID"]==0.0) &\
                (df_onlysolarwind["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

indicent_protons = df_onlysolarwind[(df_onlysolarwind["Particle_Type"]=="proton") & (df_onlysolarwind["Parent_ID"]==0.0) & \
                (df_onlysolarwind["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([indicent_electrons, indicent_protons], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=120)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.show()

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()

# Check only Photoelectrons case

In [ ]:
np.unique(df_onlyphotoemission["Volume_Name_Post"])

In [ ]:
# check the input distribution for the photons
input_dist = pd.read_csv("../sphere-charging/distributions/photon_distribution.txt",header=None,delimiter=" ")

incident_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "gamma")&(df_onlyphotoemission["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")
histdata = plt.hist(incident_gamma["Kinetic_Energy_Pre_MeV"]*1e6,bins=np.logspace(0,5,100),density=True)
print("# of incident gammas: ", len(incident_gamma))
plt.loglog(input_dist[0]*1e6,input_dist[1]/np.max(input_dist[1]),'r-')
#plt.ylim([1e-5,0.6])
plt.xlabel("Photon Energy (eV)")
plt.yscale("log")
plt.xscale("log")
#plt.xlim([0,100])

In [ ]:
## plot a distribution of the energy for every single e- that is created from photoemission ##

# get dataframe of photoelectron creation
photoelectron_creation = df_onlyphotoemission[df_onlyphotoemission["Particle_Type"] == "e-"].drop_duplicates(subset="Event_Number", keep="first")
# plot the histogram 
plt.hist(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6,bins=np.logspace(0,2,300),density=True)

print("Average photoelectron energy: ", np.mean(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6), " eV")
print("# of photoelectrons: ", len(photoelectron_creation["Kinetic_Energy_Pre_MeV"]*1e6))
# but not all of these e- are actually emitted
plt.xlabel("Photoelectron Energy (eV)")
plt.xscale("log")
plt.yscale("log")

plt.show()

In [ ]:
fig4data = pd.read_csv("Fig4-Feurerbacher1972.csv")
fig3data = pd.read_csv("Fig3-Feurerbacher1972.csv")

In [ ]:
## histogram of all the electrons that leave the material ##

# get dataframe of the last e- event within the sensitive detector
last_e_event = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "e-")].drop_duplicates(subset="Event_Number", keep="last")
# get new dataframe of all e- that left the world
world_e_energy=last_e_event[(last_e_event["Volume_Name_Post"]=="physical_cyclic") | (last_e_event["Volume_Name_Pre"]=="physical_cyclic")]

histdata = plt.hist(world_e_energy["Kinetic_Energy_Post_MeV"]*1e6,bins=np.linspace(0,35,50),density=True)

# plot thermal distribution as comparison 
energy_values = np.linspace(0,30,100)
#photoelectron_distribution = np.exp(-(energy_values)**2/1**2)*max(histdata[0])
#plt.plot(energy_values, photoelectron_distribution, "r-",label="1eV Maxwellian")
plt.plot(fig4data["x"], fig4data[" y"]/np.sum(fig4data[" y"]), 'k-',label="Literature")
plt.xlabel("Ejected Photoelectron Energy (eV)")
plt.legend()
plt.xlim([0,35])
#plt.ylim([1e-2,1])

print("Average photoelectron energy: ", np.mean(world_e_energy["Kinetic_Energy_Post_MeV"]*1e6), " eV")
# should be around 1 eV...

In [ ]:
# get all the initial gamma energy that leads to an e- escaping
matching_event_numbers = np.intersect1d(df_onlyphotoemission["Event_Number"], world_e_energy["Event_Number"])
gamma_initial_leading_to_e_ejection = df_onlyphotoemission[df_onlyphotoemission["Event_Number"].isin(matching_event_numbers)].drop_duplicates(subset="Event_Number", keep="first")

incident_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"] == "gamma")&(df_onlyphotoemission["Parent_ID"] == 0.0)].drop_duplicates(subset="Event_Number", keep="first")
incident_gamma_dist = np.histogram(incident_gamma["Kinetic_Energy_Pre_MeV"]*1e6,bins=np.logspace(0.5,3,70))

# plot thermal distribution as comparison 
pe_dist=np.histogram(gamma_initial_leading_to_e_ejection["Kinetic_Energy_Pre_MeV"]*1e6,bins=np.logspace(0.5,3,70))
print("minimum photon energy leading to photoelectric effect (eV): ", min(gamma_initial_leading_to_e_ejection["Kinetic_Energy_Pre_MeV"]*1e6))

plt.plot(incident_gamma_dist[1][:-1], pe_dist[0]/np.sum(pe_dist[0]), 'b-') #/len(incident_gamma_dist[0]
#plt.ylim([1e-5,0.6])


yield_literature = pd.read_csv("Fig1-yieldphotoelectrons.csv")
#plt.plot(yield_literature["x"], yield_literature[" y"], 'k-',label="Literature")
# plt.plot(fig4data["x"]+(8.14-5), fig4data[" y"]/np.sum(fig4data[" y"]), 'k-',label="Literature")
plt.plot(1240/(fig3data["x"]/10), fig3data[" y"]/np.sum(fig3data[" y"]), 'k-',label="Literature")
plt.xlabel("Photon Energy (eV)")
plt.axvline(x=8.1, color='k',linestyle=":",lw = 1)
plt.yscale("log")
plt.xscale("log")
plt.ylabel("Yield Per Incident Photon")
#plt.xlim([0,300])
#plt.ylim([1e-2,1])
plt.show()

In [ ]:
gamma_holes0, electrons_ejected0, electron_inside0 = calculate_stats(df_onlyphotoemission)

# plot location of where all the ion stopped in the geometry

# Create a PointCloud object
gamma_photoemission_sites = PointCloud(np.array(gamma_holes0["Pre_Step_Position_mm"].tolist()), colors=[0, 255, 0, 255])  # RGBA red points
electron_stopping_sites = PointCloud(np.array(electron_inside0["Pre_Step_Position_mm"].tolist()), colors=[0, 0, 255, 255])  # RGBA red points

# Create a scene with both mesh and points
stacked_spheres = trimesh.load_mesh('../sphere-charging/geometry/stacked_spheres_frompython.stl') 
scene = trimesh.Scene([stacked_spheres, gamma_photoemission_sites,electron_stopping_sites])

# Show scene in interactive viewer
scene.show()

In [ ]:
# plot the surface potential of the photoemission

surface,potential = plot_surface_potential_fornegativepositive_charge(electron_inside0, gamma_holes0, stacked_spheres, vmin=-0.2,vmax=0.2)

print(min(potential), max(potential))
plt.plot(potential[potential!=0])
plt.show()

# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
surface_edited.visual.vertex_colors = None
surface_edited.show()

# a lot of the surface charge ended up cancelling each other out...

In [ ]:
bbox_min = np.array([-0.25, -0.31, 0])
bbox_max = np.array([ 0.15,  0.1,  100])

# Filter vertices
in_box = np.all((surface_edited.vertices >= bbox_min) & 
                (surface_edited.vertices <= bbox_max), axis=1)

# Get face indices where all 3 vertices are in the box
face_mask = np.all(in_box[surface_edited.faces], axis=1)

# Extract submesh
cropped = surface_edited.submesh([face_mask], only_watertight=False, append=True)
cropped.show()


In [ ]:
# Define colormap and normalization
cmap = plt.cm.seismic
norm = Normalize(vmin=-0.2, vmax=0.2)

# Create a figure and a single axis for the colorbar
fig, ax = plt.subplots(figsize=(6, 1))
fig.subplots_adjust(bottom=0.5)

# Create the colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('Surface Potential (mV)')  # Optional label

plt.show()


In [ ]:
indicent_gamma = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"]=="gamma") & (df_onlyphotoemission["Parent_ID"]==0.0) & \
                (df_onlyphotoemission["Volume_Name_Post"] == "G4_SILICON_DIOXIDE")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([indicent_gamma], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=300)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
#surface_edited.visual.vertex_colors = None
surface_edited.show()

In [ ]:
# where the photoelectrons are generated 
generated_PE = df_onlyphotoemission[(df_onlyphotoemission["Particle_Type"]=="e-") & (df_onlyphotoemission["Parent_ID"]==1.0) & \
                (df_onlyphotoemission["Process_Name_Pre"] == "initStep")
].drop_duplicates(subset="Event_Number", keep="first")

merged_df = pd.concat([generated_PE], ignore_index=True)

surface,face_totals = plot_face_illumination(merged_df, stacked_spheres, vmin=0,vmax=300)
print(min(face_totals), max(face_totals))
# Make sure each triangle has its own unique vertices
surface_edited = surface.copy()
surface_edited.unmerge_vertices()
#surface_edited.visual.vertex_colors = None
surface_edited.show()